In [ ]:
import 

In [1]:
import glam
import pandas as pd
import numpy as np
import os.path


You can find the C code in this temporary file: /var/folders/np/3gkzqfkd6vddg92v1ytvrkkc0000gn/T/theano_compilation_error_8cppl48q


Exception: Compilation failed (return status=1): In file included from /Users/pradyumna/.theano/compiledir_Darwin-18.0.0-x86_64-i386-64bit-i386-3.5.4-64/lazylinker_ext/mod.cpp:1:. In file included from /Users/pradyumna/anaconda/include/python3.5m/Python.h:25:. /Users/pradyumna/anaconda/bin/../include/c++/v1/stdio.h:108:15: fatal error: 'stdio.h' file not found. #include_next <stdio.h>.               ^~~~~~~~~. 1 error generated.. 

In [2]:
import matplotlib.pyplot as plt

from scipy.stats import invgauss

import seaborn as sns
import matplotlib.pyplot as plt
plt.switch_backend('agg')

from scipy import stats

import warnings
warnings.filterwarnings('ignore')


You can find the C code in this temporary file: /var/folders/np/3gkzqfkd6vddg92v1ytvrkkc0000gn/T/theano_compilation_error_8j7tsjs3


Exception: Compilation failed (return status=1): In file included from /Users/pradyumna/.theano/compiledir_Darwin-18.0.0-x86_64-i386-64bit-i386-3.5.4-64/lazylinker_ext/mod.cpp:1:. In file included from /Users/pradyumna/anaconda/include/python3.5m/Python.h:25:. /Users/pradyumna/anaconda/bin/../include/c++/v1/stdio.h:108:15: fatal error: 'stdio.h' file not found. #include_next <stdio.h>.               ^~~~~~~~~. 1 error generated.. 

In [6]:
np.random.seed(23) # from random.org

NameError: name 'np' is not defined

In [ ]:
sufix = '_Like_FixBin5'

## Load data

In [ ]:
data = pd.read_csv(str('data/FF2018_data/GlamDataFF2018_Like_Bin_v2.csv'))
# Subset only necessary columns
data = data[['subject', 'trial', 'choice', 'rt',
         'item_value_0', 'item_value_1',
         'gaze_0', 'gaze_1']]

In [ ]:
glam_full = glam.GLAM(data)
glam_full.make_model('individual', gamma_bounds=(-1, 1), t0_val=0)

In [ ]:
glam_full.estimates = np.load(str('results/estimates/glam_FF2018_full_individual'+sufix+'.npy'))

In [3]:
#glam_full.estimates

In [4]:
# We can define n_repeats (1 by default), boundary (1 by default), error_weight ( 0.05 by default) 
#glam_full.predict(n_repeats=50)


In [5]:
## Select one participants parameters to perform the simulations
subject = 0
parameters = [glam_full.estimates[subject].get(parameter)
                              for parameter in ['v', 'gamma', 's', 'tau', 't0']]
parameters

NameError: name 'glam_full' is not defined

In [11]:
value_cols = ['item_value_{}'.format(i)
                          for i in range(glam_full.n_items)]
gaze_cols = ['gaze_{}'.format(i)
                         for i in range(glam_full.n_items)]

In [12]:
values = glam_full.data[value_cols][glam_full.data['subject'] == subject].values
gaze = glam_full.data[gaze_cols][glam_full.data['subject'] == subject].values

rt_min = glam_full.data['rt'][glam_full.data['subject'] == subject].values.min()
rt_max = glam_full.data['rt'][glam_full.data['subject'] == subject].values.max()
error_range = (rt_min, rt_max)

In [13]:
# Parameters for the simulations
n_repeats=50
boundary=1
error_weight=0.05

# Variables to run the simulation (repetitions give by n_repeats) for various trials
n_trials, n_items = values.shape
rts = np.zeros(n_trials * n_repeats) * np.nan
choices = np.zeros(n_trials * n_repeats) * np.nan
trial_idx = np.zeros(n_trials * n_repeats) * np.nan
repeat_idx = np.zeros(n_trials * n_repeats) * np.nan

running_idx = 0


In [14]:
## To run the simulations to various trials (e.g. all trials from subject s)

for trial in range(n_trials):

        for repeat in range(n_repeats):

            choice, rt = glam.simulation.simulate_trial(parameters, values[trial], gaze[trial],
                                        boundary=boundary,
                                        error_weight=error_weight,
                                        error_range=error_range)

            rts[running_idx] = rt
            choices[running_idx] = choice
            trial_idx[running_idx] = trial
            repeat_idx[running_idx] = repeat

            running_idx += 1

In [ ]:
#value_tests = np.array([[5,5],[4,4],[3,3],[2,2],[1,1],[0,0]])
#gaze_tests = np.array([[0.1,0.9],[0.25,0.75],[0.4,0.6],[0.5,0.5],[0.6, 0.4],[0.75,0.25],[0.9, 0.1]])
#gaze_tests = np.array([[0.1,0.9],[0.25,0.75],[0.5,0.5],[0.75,0.25],[0.9, 0.1]])

value_tests = np.array([[5,5],[5,2.5],[5,0]])
gaze_tests = np.array([[0.5,0.5]])

print ('gaze ratio right/left used : ' + str(gaze_tests))



# [96 ,170, 230],    # SNR
# Since SNR is calculated  as  v * SNR = s we do not consider it as free paramter

#param_tests = np.array([[-0.97, -0.57, 0.34],  # gamma
#                      [0.30, 1.62, 3.81 ]  ,    # tau
#                       [3.2e-05, 4.9e-05, 7.1e-05 ],  # v
#                       [0.0065, 0.008, 0.011]]  # s
#                      )     
           
param_tests = np.array([np.linspace(-0.97, 0.34, num = 50),  # gamma
                      np.linspace(0.30, 3.81, num = 50)  ,    # tau
                       np.linspace(3.2e-05, 7.1e-05, num = 50),  # v
                       np.linspace(0.0065, 0.011, num = 50)]  # s
                      )     
middle_point = int(len(param_tests[0])/2)
rt_min = glam_full.data['rt'][glam_full.data['subject'] == subject].values.min()
rt_max = glam_full.data['rt'][glam_full.data['subject'] == subject].values.max()
error_range = (rt_min, rt_max)
                       

parameters_names = ['$\gamma$', '$\\tau$', '$\\nu$', 's']                  

n_repeats = 500



for param_select in range(param_tests.shape[0]): # iteration for the 4 params
    
    plt.figure(figsize=(20,10));
    sns.set(style='white', font_scale=1.8)

    running_idx_params = 0
    mean_rts_params = np.zeros((value_tests.shape[0] ,param_tests.shape[1])) * np.nan
    choice_params = np.zeros((value_tests.shape[0],param_tests.shape[1])) * np.nan

    for param_instance_select in range(param_tests.shape[1]): # iteration for (currently) 3 instances (min, median, max) for each parameter
        # we will vary on parameter at a time, we will mantain the rest of paramters constant at the middle level.
    
        gamma = param_tests[0][middle_point]
        tau = param_tests[1][middle_point]
        v = param_tests[2][middle_point]
        s = param_tests[3][middle_point]
        t0 = 0
        
        if param_select == 0:  #gamma
                    gamma = param_tests[0][param_instance_select]
        if param_select == 1:  #tau
                    tau = param_tests[1][param_instance_select]
        if param_select == 2:  #v
                    v = param_tests[2][param_instance_select]
        if param_select == 3:  #s
                    s = param_tests[3][param_instance_select]
        
        parameters = [v , gamma, s, tau, t0]
        
        
        
        
        running_idx_value = 0
        for trial in range(n_trials):

            
                n_trials, n_items = value_tests.shape

                for gaze_select in range(gaze_tests.shape[0]):
                
                    rts = np.zeros(n_repeats) * np.nan
                    choices = np.zeros(n_repeats) * np.nan
                    trial_idx = np.zeros(n_repeats) * np.nan
                    repeat_idx = np.zeros(n_repeats) * np.nan
                    running_idx = 0
                
                    for repeat in range(n_repeats):
                            
                            value_trial =value_tests[trial]
                            gaze_trial =gaze_tests[gaze_select]
                            
                            choice, rt = glam.simulation.simulate_trial(parameters, value_trial, gaze_trial,
                                                        boundary=boundary,
                                                        error_weight=error_weight,
                                                        error_range=error_range)
                
                            rts[running_idx] = rt
                            choices[running_idx] = choice
                            trial_idx[running_idx] = trial
                            repeat_idx[running_idx] = repeat
                            
                            running_idx += 1
                            
                       
                    # Calculate mean value for reaction times        
                    mean_rts =  rts.mean()              
                    mean_choice =  choices.sum()/choices.size  # choice left : 0 ; choice right : 1   
                    # Add the mean value for this parameters interation               
                    mean_rts_params [running_idx_value][running_idx_params] = mean_rts
                    choice_params [running_idx_value][running_idx_params] = mean_choice

                    running_idx_value += 1 

        
        running_idx_params += 1 
                   
    plt.subplot(121)        
    plt.plot(param_tests[param_select], mean_rts_params[0] , marker='o', markersize=5, color="red");
    plt.plot(param_tests[param_select], mean_rts_params[1] , marker='o', markersize=5, color="blue");
    plt.plot(param_tests[param_select], mean_rts_params[2] , marker='o', markersize=5, color="green");

    plt.title("RT across parameters."+ "Parameter:  " + parameters_names[param_select]);
    plt.ylabel('Reaction Time');
    plt.xlabel('Parameter ' + parameters_names[param_select] );
    plt.legend((str(value_tests[0]), str(value_tests[1]),str(value_tests[2])),loc='upper right', title = 'Value items');
    plt.subplot(122)
    plt.plot(param_tests[param_select], choice_params[0] , marker='o', markersize=5, color="red");
    plt.plot(param_tests[param_select], choice_params[1] , marker='o', markersize=5, color="blue");
    plt.plot(param_tests[param_select], choice_params[2] , marker='o', markersize=5, color="green");
    plt.title("Proportion of right answers across params."+ "Parameter: " + parameters_names[param_select]);
    plt.ylabel('Choice');
    plt.xlabel('Parameter ' + parameters_names[param_select] );
    plt.legend((str(value_tests[0]), str(value_tests[1]),str(value_tests[2])),loc='upper right', title = 'Value items');
